In [2]:
require 'unsup'

true	


In [3]:
require 'nn';

In [4]:
require 'gnuplot'

true	


In [5]:
require 'Atan2'

true	


In [6]:
require 'nngraph'

true	


In [7]:
n_hidden = 10
n_input = 100
input = torch.rand(100,2)

In [8]:
x = torch.cmul(input[{{},1}],torch.cos(input[{{},2}]))
y = torch.cmul(input[{{},1}],torch.sin(input[{{},2}]))
input = torch.cat(x,y,2)

In [68]:
enc = nn.Sequential()
encpar = nn.ParallelTable()

--imaginary activation part
img = nn.Sequential()

par1 = nn.Parallel(2,2)

linear1 = nn.Linear(n_input,n_hidden)
linear2 = nn.Linear(n_input,n_hidden)
linear1:share(linear2,'weight')

cont1 = nn.Sequential()
cont1:add(linear1) -- w*x
cont1:add(nn.Reshape(10,1)) --reshape for concat
cont2 = nn.Sequential()
cont2:add(linear2) -- w*y
cont2:add(nn.Reshape(10,1)) --reshape for concat
par1:add(cont1)
par1:add(cont2)

img:add(par1)
img:add(nn.Replicate(2)) --duplicate for phase calculations
img:add(nn.SplitTable(1))

partable = nn.ParallelTable()

seq1 = nn.Sequential()
par2 = nn.Parallel(2,2)

cont3 = nn.Sequential()
cont3:add(nn.Square()) -- (wx)^2
cont3:add(nn.Reshape(10,1)) 
cont4 = nn.Sequential() 
cont4:add(nn.Square()) -- (wx)^2
cont4:add(nn.Reshape(10,1))

par2:add(cont3)
par2:add(cont4)

seq1:add(par2)
seq1:add(nn.SplitTable(2)) 
seq1:add(nn.CAddTable()) -- (wx)^2 + (wy)^2
seq1:add(nn.Sqrt()) 
seq1:add(nn.MulConstant(0.5,true))

partable:add(seq1)

partable:add(nn.Atan2()) -- atan(wy,wx)

img:add(partable)


--real activation part
real = nn.Sequential()

par2 = nn.ParallelTable()

par2:add(nn.Square())
par2:add(nn.Square()) 

real:add(nn.SplitTable(2))
real:add(par2)
real:add(nn.CAddTable())  -- x^2 + y^2
real:add(nn.Sqrt())
linear3 = nn.Linear(n_input,n_hidden)
linear3:share(linear1,'weight')
real:add(linear3)
real:add(nn.MulConstant(0.5,true))

--
encpar:add(img)
encpar:add(real)

enc:add(encpar)
enc:add(nn.FlattenTable())

-------------------------------------------------
join = nn.Sequential()
join:add(nn.JoinTable(1))
join:add(nn.Reshape(3,10))

--par3 = nn.ParallelTable()
cont5 = nn.Concat(2)

--select first column, i.e. phase
reshape1 = nn.Sequential()
reshape1:add(nn.Select(1,1))
reshape1:add(nn.Reshape(10,1))
cont5:add(reshape1)

--select second and third column and add them up
cont6 = nn.Sequential()
partable2 = nn.Concat(2)
reshape2 = nn.Sequential()
reshape2:add(nn.Select(1,2))                       --Magic in order to seperate and rejoin table
reshape2:add(nn.Reshape(10,1))                     --don't try to understand
partable2:add(reshape1)
reshape3 = nn.Sequential()
reshape3:add(nn.Select(1,3))
reshape3:add(nn.Reshape(10,1))
partable2:add(reshape1)
cont6:add(partable2)
cont6:add(nn.SplitTable(2))
cont6:add(nn.CAddTable())
cont6:add(nn.Reshape(10,1))

cont5:add(cont6)

enc:add(join)
enc:add(cont5)
enc:add(nn.SplitTable(2))
-------------------------------------------------

sigmoid = nn.ParallelTable()
sigmoid:add(nn.Identity())
sigmoid:add(nn.Sigmoid())

enc:add(sigmoid)
res = enc:forward({input,input}) -- Outputs a table with phase in res[1] and activation in res[2]
print(res)
print(enc:backward({input,input},res))



{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 10
}
{
  1 : DoubleTensor - size: 100x2
  2 : DoubleTensor - size: 100x2
}


In [ ]:
criterion = nn.MultiCriterion()

xcrit = nn.MSECriterion()
ycrit = nn.MSECriterion()


dec = nn.Sequential()
dec:add(nn.Linear(n_hidden,n_input))

dec:get(1).weight = r1:get(1).weight:t()
dec:get(1).gradWeight = r1:get(1).gradWeight:t()

autoencoder = nn.Sequential()
autoencoder:add(enc)
autoencoder:add(dec)

In [11]:
a = {input,input}
result = autoencoder:forward(a)
gradient = autoencoder:backward(a,result)
print(gradient)

{
  1 : DoubleTensor - size: 100x2
  2 : DoubleTensor - size: 100x2
}


In [12]:
print(autoencoder:parameters())

In [13]:
for i,module in ipairs(autoencoder:listModules()) do
   print(module)
end

{
  1 : DoubleTensor - size: 10x100
  2 : DoubleTensor - size: 10
  3 : DoubleTensor - size: 10x100
  4 : DoubleTensor - size: 10
  5 : DoubleTensor - size: 10x100
  6 : DoubleTensor - size: 10
  7 : DoubleTensor - size: 100x10
  8 : DoubleTensor - size: 100
}
{
  1 : DoubleTensor - size: 10x100
  2 : DoubleTensor - size: 10
  3 : DoubleTensor - size: 10x100
  4 : DoubleTensor - size: 10
  5 : DoubleTensor - size: 10x100
  6 : DoubleTensor - size: 10
  7 : DoubleTensor - size: 100x10
  8 : DoubleTensor - size: 100
}
nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> output]
    (1): nn.ParallelTable {
      input
        |`-> (1): nn.Sequential {
        |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
        |      (1): nn.SplitTable
        |      (2): nn.ParallelTable {
        |        input
        |          |`-> (1): nn.Sequential {
        |          |      [input -> (1) -> (2) -> output]
        |          | 

        nn.Sequential {
          [input -> (1) -> (2) -> (3) -> output]
          (1): nn.ParallelTable {
            input
              |`-> (1): nn.Sequential {
              |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
              |      (1): nn.SplitTable
              |      (2): nn.ParallelTable {
              |        input
              |          |`-> (1): nn.Sequential {
              |          |      [input -> (1) -> (2) -> output]
              |          |      (1): nn.Linear(100 -> 10)
              |          |      (2): nn.Square
              |          |    }
              |          |`-> (2): nn.Sequential {
              |          |      [input -> (1) -> (2) -> output]
              |          |      (1): nn.Linear(100 -> 10)
              |          |      (2): nn.Square
              |          |    }
              |           ... -> output
              |      }
              |      (3): nn.CAddTable
              |      (4): nn.Sqrt
       


                }
              3 : 
                nn.Sigmoid
                {
                  gradInput : DoubleTensor - size: 10
                  output : DoubleTensor - size: 10
                }
            }
          output : DoubleTensor - size: 10
        }
      2 : 
        nn.Sequential {
          [input -> (1) -> output]
          (1): nn.Linear(10 -> 100)
        }
        {
          gradInput : DoubleTensor - size: 10
          modules : 
            {
              1 : 
                nn.Linear(10 -> 100)
                {
                  gradBias : DoubleTensor - size: 100
                  weight : DoubleTensor - size: 100x10
                  bias : DoubleTensor - size: 100
                  gradInput : DoubleTensor - size: 10
                  gradWeight : DoubleTensor - size: 100x10
                  output : DoubleTensor - size: 100
                }
            }
          output : DoubleTensor - size: 100
        }
    }
  output : DoubleTensor - size

ubleTensor - size: 100x2
              2 : DoubleTensor - size: 100x2
            }
          modules : 
            {
              1 : 
                nn.Sequential {
                  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
                  (1): nn.SplitTable
                  (2): nn.ParallelTable {
                    input
                      |`-> (1): nn.Sequential {
                      |      [input -> (1) -> (2) -> output]
                      |      (1): nn.Linear(100 -> 10)
                      |      (2): nn.Square
                      |    }
                      |`-> (2): nn.Sequential {
                      |      [input -> (1) -> (2) -> output]
                      |      (1): nn.Linear(100 -> 10)
                      |      (2): nn.Square
                      |    }
                       ... -> output
                  }
                  (3): nn.CAddTable
                  (4): nn.Sqrt
                  (5): nn.MulConstant
                }



    }
  output : DoubleTensor - size: 10
}
nn.ParallelTable {
  input
    |`-> (1): nn.Sequential {
    |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
    |      (1): nn.SplitTable
    |      (2): nn.ParallelTable {
    |        input
    |          |`-> (1): nn.Sequential {
    |          |      [input -> (1) -> (2) -> output]
    |          |      (1): nn.Linear(100 -> 10)
    |          |      (2): nn.Square
    |          |    }
    |          |`-> (2): nn.Sequential {
    |          |      [input -> (1) -> (2) -> output]
    |          |      (1): nn.Linear(100 -> 10)
    |          |      (2): nn.Square
    |          |    }
    |           ... -> output
    |      }
    |      (3): nn.CAddTable
    |      (4): nn.Sqrt
    |      (5): nn.MulConstant
    |    }
    |`-> (2): nn.Sequential {
    |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
    |      (1): nn.SplitTable
    |      (2): nn.ParallelTable {
    |        input
    |          |`-> (1):

...

          }
            }
          output : DoubleTensor - size: 10
        }
    }
  output : 
    {
      1 : DoubleTensor - size: 10
      2 : DoubleTensor - size: 10
    }
}
nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.Linear(100 -> 10)
  (2): nn.Square
}
{
  gradInput : DoubleTensor - size: 100
  modules : 
    {
      1 : 
        nn.Linear(100 -> 10)
        {
          gradBias : DoubleTensor - size: 10
          weight : DoubleTensor - size: 10x100
          bias : DoubleTensor - size: 10
          gradInput : DoubleTensor - size: 100
          accUpdateGradParameters : function: 0x41452cf8
          gradWeight : DoubleTensor - size: 10x100
          output : DoubleTensor - size: 10
        }
      2 : 
        nn.Square
        {
          gradInput : DoubleTensor - size: 10
          output : DoubleTensor - size: 10
        }
    }
  output : DoubleTensor - size: 10
}
nn.Linear(100 -> 10)
{
  gradBias : DoubleTensor - size: 10
  weight : DoubleTensor - size: 


        }
      2 : 
        nn.Square
        {
          gradInput : DoubleTensor - size: 10
          output : DoubleTensor - size: 10
        }
    }
  output : DoubleTensor - size: 10
}
nn.Linear(100 -> 10)
{
  gradBias : DoubleTensor - size: 10
  weight : DoubleTensor - size: 10x100
  bias : DoubleTensor - size: 10
  gradInput : DoubleTensor - size: 100
  accUpdateGradParameters : function: 0x41452cf8
  gradWeight : DoubleTensor - size: 10x100
  output : DoubleTensor - size: 10
}
nn.Square
{
  gradInput : DoubleTensor - size: 10
  output : DoubleTensor - size: 10
}
nn.CAddTable
{
  gradInput : 
    {
      1 : DoubleTensor - size: 10
      2 : DoubleTensor - size: 10
    }
  output : DoubleTensor - size: 10
}
nn.Sqrt
{
  gradInput : DoubleTensor - size: 10
  eps : 0
  output : DoubleTensor - size: 10
}
nn.MulConstant
{
  gradInput : DoubleTensor - size: 10
  inplace : true
  constant_scalar : 0.5
  output : DoubleTensor - size: 10
}
nn.Sequential {
  [input -> (1) -> (2) -> (3) 

    1 : 
                nn.Square
                {
                  gradInput : DoubleTensor - size: 100
                  output : DoubleTensor - size: 100
                }
              2 : 
                nn.Square
                {
                  gradInput : DoubleTensor - size: 100
                  output : DoubleTensor - size: 100
                }
            }
          output : 
            {
              1 : DoubleTensor - size: 100
              2 : DoubleTensor - size: 100
            }
        }
      3 : 
        nn.CAddTable
        {
          gradInput : 
            {
              1 : DoubleTensor - size: 100
              2 : DoubleTensor - size: 100
            }
          output : DoubleTensor - size: 100
        }
      4 : 
        nn.Sqrt
        {
          gradInput : DoubleTensor - size: 100
          eps : 0
          output : DoubleTensor - size: 100
        }
      5 : 
        nn.Linear(100 -> 10)
        {
          gradBias : DoubleTensor - s

 {
      1 : DoubleTensor - size: 100
      2 : DoubleTensor - size: 100
    }
}
nn.ParallelTable {
  input
    |`-> (1): nn.Square
    |`-> (2): nn.Square
     ... -> output
}
{
  gradInput : 
    {
      1 : DoubleTensor - size: 100
      2 : DoubleTensor - size: 100
    }
  modules : 
    {
      1 : 
        nn.Square
        {
          gradInput : DoubleTensor - size: 100
          output : DoubleTensor - size: 100
        }
      2 : 
        nn.Square
        {
          gradInput : DoubleTensor - size: 100
          output : DoubleTensor - size: 100
        }
    }
  output : 
    {
      1 : DoubleTensor - size: 100
      2 : DoubleTensor - size: 100
    }
}
nn.Square
{
  gradInput : DoubleTensor - size: 100
  output : DoubleTensor - size: 100
}
nn.Square
{
  gradInput : DoubleTensor - size: 100
  output : DoubleTensor - size: 100
}
nn.CAddTable
{
  gradInput : 
    {
      1 : DoubleTensor - size: 100
      2 : DoubleTensor - size: 100
    }
  output : DoubleTensor - size: 1

        }
    }
  output : DoubleTensor - size: 100
}
nn.Linear(10 -> 100)
{
  gradBias : DoubleTensor - size: 100
  weight : DoubleTensor - size: 100x10
  bias : DoubleTensor - size: 100
  gradInput : DoubleTensor - size: 10
  gradWeight : DoubleTensor - size: 100x10
  output : DoubleTensor - size: 100
}
